Import necessary files and libs:

In [3]:
import pandas as pd
import requests as r
from files.lol_regions import regions
from files.lol_genders import genders
from files.lol_names import correction_roles

### __Champion gender:__

Create dataframe:

In [4]:
lol_df = pd.read_csv("files/data.csv")

Add gender to respective champions:

In [5]:
for k,v in genders.items():
    lol_df.loc[lol_df["champion"] == k, "gender"] = v

In [ ]:
lol_df

### __Champion roles:__

Get list from API of all champions:

In [7]:
patch = "12.12.1"
ddragon = r.get(f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json").json()
champions = list(ddragon['data'].keys())

Create and format list with champion roles:

In [8]:
list_tags = []
for c in champions:
    tag_champion = ddragon['data'][c]['tags']
    try:
        list_tags.append([c, f"{tag_champion[0]}, {tag_champion[1]}"])
    except:
        list_tags.append([c, f"{tag_champion[0]}"])

Create dataframe and apply name correction:

In [9]:
tags_df = pd.DataFrame(list_tags, columns=["champion", "role"])
for key, value in correction_roles.items():
    tags_df.loc[tags_df["champion"] == key, "champion"] = value

Merge dataframes:

In [10]:
lol_df = lol_df.merge(tags_df)

In [ ]:
lol_df

### __Scrapping Stats with Selenium:__

**Note:**
- you need to install Selenium and Webdriver to proceed.
- Selenium will emulate an automation browser in your computer.

#### __Get HTML in two different ways:__

##### __● If you're running this file in Google Collab:__

In [ ]:
pip install kora

In [ ]:
from kora.selenium import wd
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
wd.get("https://na.op.gg/statistics/champions?hl=en_US&region=global")
time.sleep(7) # Wait time to pull html
html = wd.page_source

##### __● If you're running this file in Linux:__

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [15]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://na.op.gg/statistics/champions?hl=en_US&region=global"

driver.get(url)
time.sleep(7) # Wait time to pull html
html = driver.page_source
driver.close()

##### __Let's continue:__

Transform html into dataframe:

In [17]:
stats_df = pd.read_html(html)
stats_df = stats_df[1]

Clean and organize column names:

In [18]:
stats_df.drop(["#", "CS", "Gold", "Games played", "KDA"], inplace=True, axis=1, errors='ignore')
stats_df.rename(
inplace=True,
columns= {"Champion": "champion", 
          "Win rate": "win_rate",
          "Pick ratio per game": "pick_rate",
          "Ban ratio per game": "ban_rate"})
          
stats_df.sort_values("champion", inplace=True, axis=0, ignore_index=True)

Apply correction in champion names:

In [19]:
main_champions = pd.DataFrame(lol_df.champion.unique())
main_champions.rename(columns={0:"champion"}, inplace=True)

stats_df.drop("champion", inplace=True, axis=1)
stats_df = pd.concat([main_champions,stats_df], axis=1)

Merge dataframes:

In [20]:
lol_df = lol_df.merge(stats_df)

In [ ]:
stats_df

In [ ]:
lol_df